# Read train data and test data

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_validate
from sklearn.ensemble import ExtraTreesClassifier, IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFECV, SelectFromModel, VarianceThreshold
from itertools import compress

In [ ]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ read data sets from files ^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


#read data sets from files
data = pd.read_csv(r"./traindata.csv")
testdata = pd.read_csv(r"./testdata.csv")

# parameters tuning for Decision Tree

In [ ]:
# first round

In [3]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
from sklearn.tree import DecisionTreeClassifier
mm = MinMaxScaler()
#mm = StandardScaler()

selector = VarianceThreshold(np.median(data_aftcat.var().values))
model = DecisionTreeClassifier(random_state=10, criterion='gini', max_depth=18, max_features=25, min_impurity_decrease=0.0, min_samples_leaf=1, splitter='best')
    
#model = RandomForestClassifier(random_state = 90, min_samples_split=2,n_estimators=61,max_depth=24, max_features=27,min_samples_leaf=1, n_jobs=-1)
#model = GradientBoostingClassifier(max_features=3,learning_rate=0.1,n_estimators=130,min_samples_split=100,min_samples_leaf=7,max_depth=15,random_state = 10)
#model = MLPClassifier(random_state=1, max_iter=10000,hidden_layer_sizes = (160,160), activation='tanh',solver='adam')

print("DT")

pipe_steps = [('mm',mm),('selector',selector),('model',model)]
id_pipeline = Pipeline(steps=pipe_steps)

# evaluate the pipeline using the crossvalidation technique defined in cv

# ------------------- score ------------------------------------
def score_mean_std(score_sf):
    index_a1 = []
    index_a2 = []

    for i in range(score_sf.shape[1]):
        index_a1.append(np.average(score_sf[score_sf.columns[i]]))
        index_a2.append(np.std(score_sf[score_sf.columns[i]]))

    index_a1 = pd.DataFrame(index_a1, index = score_sf.columns)
    index_a2 = pd.DataFrame(index_a2, index = score_sf.columns)
    #print(index_a1.T)
    #print(index_a2.T)
    re = pd.concat([index_a1.T,index_a2.T],axis=0)
    re.index = ["avg","std"]
    #print(re)
    return re


scoring = {'f1', 'precision', 'accuracy',
           'recall', 'roc_auc'}
score_sf = cross_validate(id_pipeline, data_aftcat, data_y.values.ravel(),cv=10,scoring=scoring, n_jobs=-1)
score_sf_re = score_mean_std(pd.DataFrame(score_sf))

print("score_sf_re: \n", pd.DataFrame(score_sf_re))



DT
score_sf_re: 
      fit_time  score_time  test_recall   test_f1  test_accuracy  test_roc_auc  \
avg  0.456422    0.050535     0.996546  0.995731       0.995913      0.995940   
std  0.127955    0.014549     0.001136  0.000957       0.000916      0.000917   

     test_precision  
avg        0.994919  
std        0.001159  


In [8]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Prediction  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# step 1: use train data MinMaxScaler() fit to transform test data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

tmm = MinMaxScaler().fit(data_aftcat)
result_train = tmm.transform(data_aftcat)
X_train = pd.DataFrame(result_train)
X_train.columns = tmm.get_feature_names_out()
#print("X_train",X_train.info())


result_test = tmm.transform(testdata_aftcat)
X_test = pd.DataFrame(result_test)
X_test.columns = tmm.get_feature_names_out()
#print("X_test",X_test.info())

# step 2: use train data VarianceThreshold() fit to transform test data
ss = selector.fit(data_aftcat)
newdata = pd.DataFrame(ss.transform(X_train))
newdata.columns = ss.get_feature_names_out()  
#print("newddata:",newdata.head())
                       
newtestdata = pd.DataFrame(ss.transform(X_test))
newtestdata.columns = ss.get_feature_names_out()
#print("newtestdata:",newtestdata.head())

# step 3: prediction
model.fit(newdata,data_y.values.ravel())

y_pre = model.predict(newtestdata).astype(int)
print("sum:",y_pre.sum())

pre = pd.DataFrame(columns=["ID","Class"])
pre["ID"] = range(y_pre.shape[0])
pre["Class"] = y_pre
pre.to_csv(r'./DT-pre.csv',index=None)

sum: 10216


# round 2

In [17]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
from sklearn.tree import DecisionTreeClassifier
mm = MinMaxScaler()
#mm = StandardScaler()

selector = VarianceThreshold(np.median(data_aftcat.var().values))
model = DecisionTreeClassifier(random_state=10, criterion='gini', max_depth=12, max_features=25, min_impurity_decrease=0.0, min_samples_leaf=1, splitter='best')
print("DT")

pipe_steps = [('mm',mm),('selector',selector),('model',model)]
id_pipeline = Pipeline(steps=pipe_steps)

# evaluate the pipeline using the crossvalidation technique defined in cv

# ------------------- score ------------------------------------
def score_mean_std(score_sf):
    index_a1 = []
    index_a2 = []

    for i in range(score_sf.shape[1]):
        index_a1.append(np.average(score_sf[score_sf.columns[i]]))
        index_a2.append(np.std(score_sf[score_sf.columns[i]]))

    index_a1 = pd.DataFrame(index_a1, index = score_sf.columns)
    index_a2 = pd.DataFrame(index_a2, index = score_sf.columns)
    #print(index_a1.T)
    #print(index_a2.T)
    re = pd.concat([index_a1.T,index_a2.T],axis=0)
    re.index = ["avg","std"]
    #print(re)
    return re


scoring = {'f1', 'precision', 'accuracy',
           'recall', 'roc_auc'}
score_sf = cross_validate(id_pipeline, data_aftcat, data_y.values.ravel(),cv=10,scoring=scoring, n_jobs=-1)
score_sf_re = score_mean_std(pd.DataFrame(score_sf))

print("score_sf_re: \n", pd.DataFrame(score_sf_re))




DT
score_sf_re: 
      fit_time  score_time  test_recall   test_f1  test_accuracy  test_roc_auc  \
avg  0.368750    0.052989      0.99600  0.995277       0.995479      0.996239   
std  0.065035    0.018917      0.00142  0.001269       0.001217      0.001463   

     test_precision  
avg        0.994559  
std        0.002171  


In [18]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Prediction  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# step 1: use train data MinMaxScaler() fit to transform test data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

tmm = MinMaxScaler().fit(data_aftcat)
result_train = tmm.transform(data_aftcat)
X_train = pd.DataFrame(result_train)
X_train.columns = tmm.get_feature_names_out()
#print("X_train",X_train.info())


result_test = tmm.transform(testdata_aftcat)
X_test = pd.DataFrame(result_test)
X_test.columns = tmm.get_feature_names_out()
#print("X_test",X_test.info())

# step 2: use train data VarianceThreshold() fit to transform test data
ss = selector.fit(data_aftcat)
newdata = pd.DataFrame(ss.transform(X_train))
newdata.columns = ss.get_feature_names_out()  
#print("newddata:",newdata.head())
                       
newtestdata = pd.DataFrame(ss.transform(X_test))
newtestdata.columns = ss.get_feature_names_out()
#print("newtestdata:",newtestdata.head())

# step 3: prediction
model.fit(newdata,data_y.values.ravel())

y_pre = model.predict(newtestdata).astype(int)
print("sum:",y_pre.sum())

pre = pd.DataFrame(columns=["ID","Class"])
pre["ID"] = range(y_pre.shape[0])
pre["Class"] = y_pre
pre.to_csv(r'./DT-pre.csv',index=None)

sum: 10289


# Round 1 - DT   max_depth

In [24]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
from sklearn.tree import DecisionTreeClassifier
mm = MinMaxScaler()
#mm = StandardScaler()

data_aftcat_mm = MinMaxScaler().fit_transform(data_aftcat, data_y.values.ravel())

selector = VarianceThreshold(np.median(data_aftcat.var().values))
#model = DecisionTreeClassifier(random_state=10, criterion='gini', max_depth=12, max_features=25, min_impurity_decrease=0.0, min_samples_leaf=1, splitter='best')
#model = DecisionTreeClassifier(random_state=10)



te = []
for i in range(20):
    model = DecisionTreeClassifier(random_state=25,max_depth=i+1) 
    pipe_steps = [('mm',mm),('selector',selector),('model',model)]
    id_pipeline = Pipeline(steps=pipe_steps)
    
    score_sf = cross_validate(id_pipeline, data_aftcat, data_y.values.ravel(),cv=10,scoring='f1', n_jobs=-1)
    #print("score_sf:", score_sf['test_score'])
    score_te = np.average(pd.DataFrame(score_sf['test_score']))
    te.append(score_te)

print(max(te),te.index(max(te)))

0.9959105326468153 18


In [ ]:
# Round 2 - DT 

In [29]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

mm = MinMaxScaler()
#mm = StandardScaler()

selector = VarianceThreshold(np.median(data_aftcat.var().values))

from sklearn.model_selection import GridSearchCV


parameters = {'splitter':('best','random')
             ,'criterion':("gini","entropy")
             ,"max_depth":[*range(2,20)]
             ,'min_samples_leaf':[*range(1,4,3)]
             ,'max_features':[*range(1,40,1)]
             }
model = DecisionTreeClassifier(random_state=25) 
pipe_steps = [('mm',mm),('selector',selector),('model',model)]
id_pipeline = Pipeline(steps=pipe_steps)
#score_sf = cross_validate(id_pipeline, data_aftcat, data_y.values.ravel(),cv=10,scoring='f1', n_jobs=-1)

GS = GridSearchCV(model, parameters, cv=10, scoring='f1')
GS.fit(data_aftcat, data_y.values.ravel())
print(GS.best_score_)
print(GS.best_params_)

0.9975893951031413
{'criterion': 'gini', 'max_depth': 14, 'max_features': 37, 'min_samples_leaf': 1, 'splitter': 'best'}


###### model = DecisionTreeClassifier(random_state=25, criterion='gini', max_depth = 14, max_features=37, min_samples_leaf=1, splitter='best') 

In [ ]:
# Round 3 - DT

In [ ]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

mm = MinMaxScaler()
#mm = StandardScaler()

selector = VarianceThreshold(np.median(data_aftcat.var().values))

from sklearn.model_selection import GridSearchCV


parameters = {'splitter':('best','random')
             ,'criterion':("gini","entropy")
             ,"max_depth":[*range(2,20)]
             ,'min_samples_leaf':[*range(1,4,3)]
             ,'max_features':[*range(1,40,1)]
             }
model = DecisionTreeClassifier(random_state=25, criterion='gini', max_depth = 14, max_features=37, min_samples_leaf=1, splitter='best') 
pipe_steps = [('mm',mm),('selector',selector),('model',model)]
id_pipeline = Pipeline(steps=pipe_steps)
#score_sf = cross_validate(id_pipeline, data_aftcat, data_y.values.ravel(),cv=10,scoring='f1', n_jobs=-1)

GS = GridSearchCV(model, parameters, cv=10, scoring='f1')
GS.fit(data_aftcat, data_y.values.ravel())
print(GS.best_score_)
print(GS.best_params_)